Group Name: AG 13

Student Name (Student ID):

1. Zhao Yufan (A0255971U)

2. Nayanthara Prathap Menon (A0261939R)

3. Valeriy Ivanov (A0228607B)

# Question 1

Consider the maze shown below. The Maze has 16 rows and 24 columns The objective is to find a shortest path from cell $S$ to cell $G$.


![Maze](Maze_Assignment_1.jpg)


The agent can take four actions in each cell: 'RIGHT', 'DOWN', 'UP', 'LEFT'.  

Each cell is represented as $(x,y)$, where $x$ indicates row number and $y$ indicates column number. Action 'UP' takes the agent from cell $(x,y)$ to $(x+1,y)$. Action 'DOWN' takes the agent from cell $(x,y)$ to $(x-1,y)$. Action 'RIGHT' takes the agent from cell $(x,y)$ to $(x,y+1)$. Action 'LEFT' takes the agent from cell $(x,y)$ to $(x,y-1)$. The triplet $(s,a,s')$  indicates that taking action $a$ at state $s$ leads to state $s'$. Actions 'LEFT' or 'RIGHT' cost 10 units for all $(s,a,s')$. Actions 'UP' or 'DOWN' cost 1 unit for all  $(s,a,s')$.  The agent cannot move into cells that are shaded. Assume that the agent knows the boundaries of the maze and has full observability. Consequently, at the bottom (row 0) and top (row 15), the agent will not take actions 'DOWN' and 'UP', respectively; at left (column 0) and right (column 23) columns, the agent will not take 'LEFT' and 'RIGHT' actions, respectively. Similalry, the agent will not take actions that lead to shaded region in the maze.

## **Q1.a: Class Maze(Problem)** [3 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms. 

## **Q1.b: Analysis of the Algorithms** [7 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

    g. Any other variants for search algorithms that are not discussed in the class (bonus/optional question) 

2. Identify the number of nodes generated, number of nodes expanded, maximum frontier size, and path-cost for the above algorithms. 
 
3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.
 
4. Draw a bar plot comparing the statistics of the algorithms and explain the results. 

Note 1: You must follow the problem formulation discussed in the class. A abstract class for Problem amd Node definition is presented below. The search tree generation should follow the template discussed in the class (i.e., Node class, expand methods, etc.). 

Note 2: If you are borrowing a block of code (for example, helper functions or data structures, etc.) from AIMA4e repository, you have to acknowledge it in the code. 

Note 3: The code should be written in a single jupyter notebook file.

$\textbf{All helper functions, templates, and data structure implementations were taken from the AIMA4e python repository.}$

In [180]:
import sys
from collections import deque
import math

class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [181]:
# Use the following Node class to generate search tree
import math
class Node:

    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost = 0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)
    
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))

    def __lt__(self, node):
        return self.path_cost < node.path_cost

    def expand(self , problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state,parent=self,action=action,path_cost=problem.action_cost(self.state , action, next_state)+self.path_cost)

        return next_node
    
    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        return hash(self.state)

In [182]:
def is_in(elt, seq):
#"""Similar to (elt in seq), but compares with 'is', not '=='."""
  return any(x is elt for x in seq)

In [183]:
class Maze(Problem):
    #Your code goes here
    #We are implementing a maze with an agent inside, with specified starting position and goal
    def __init__(self , width: int  , height: int , initial: tuple , goal: tuple):
        self.width = width
        self.height = height
        self.walls = []  #the coordinates of the unpassable squares
        self.initial = initial
        self.pos = initial #current position of agent
        self.goal = goal
    
    def actions(self, state: tuple):
        actions = {'UP' , 'DOWN' , 'RIGHT' , 'LEFT'}
        
        #remove moves that moves agent out of maze boundary
        if state[0] == self.width:
            actions.remove('RIGHT')
        if state[0] == 0:
            actions.remove('LEFT')        
        if state[1] == self.height:
            actions.remove('UP')
        if state[1] == 0:
            actions.remove('DOWN')
            
        #remove moves that moves agent into the walls
        if (state[0] - 1 , state[1]) in self.walls:
            actions.remove('LEFT')  
        if (state[0] + 1 , state[1]) in self.walls:
            actions.remove('RIGHT')
        if (state[0] , state[1] - 1) in self.walls:
            actions.remove('DOWN')
        if (state[0] , state[1] + 1) in self.walls:
            actions.remove('UP')   
            
        #remove all moves if we somehow end up in the wall
        if (state[0] , state[1]) in self.walls:
            actions.remove('UP' , 'DOWN' , 'RIGHT' , 'LEFT')      
        
        return actions
    
    
    
    def result(self, state, action):
        if action == 'UP':
            return (state[0],state[1] + 1)
        elif action == 'DOWN':
            return (state[0],state[1] - 1)
        elif action == 'LEFT':
            return (state[0] - 1,state[1])
        elif action == 'RIGHT':
            return (state[0] + 1,state[1])
        
    def action_cost(self, s, a, s1): 
        if a == 'LEFT' or a == 'RIGHT':
            return 10
        elif a == 'UP' or a == 'DOWN':
            return 1


    def goal_test(self, state):
    #"""Return True if the state is a goal. The default method compares the
    #state to self.goal or checks for state in self.goal if it is a
    #list, as specified in the constructor. Override this method if
    #checking against a single self.goal is not enough."""
      if isinstance(self.goal, list):
          return is_in(state, self.goal)
      else:
          return state == self.goal

    
    #By inspection, the shortest path has length 12
    #The path is hugging the bottom side of the wall and going around it
    
    #First Heuristic based on L1Norm, or Taxicab distance from state to goal
    def h1(self, node):
        
        
        def L1Norm(a,b):
            if len(a) != len(b):
                raise ValueError('dimension mismatch')
                
            distance = 10*abs(a[0] - b[0]) + abs(a[1] - b[1])
            
            return distance
        
        return L1Norm(node.state , self.goal)
    
    
    #Second Heuristic based on L Infinity Norm, or max absolute difference in the coordinates
    
    def h2(self, node):
        
        def LInfNorm(a,b):
        
            if len(a) != len(b):
                raise ValueError('dimension mismatch')
            
        
            distance = max(10*abs(a[0] - b[0]) , abs(a[1] - b[1]))
                
            return distance
        
        return LInfNorm(node.state , self.goal)

In [184]:
def breadth_first_tree_search(problem):


    frontier = deque([Node(problem.initial, parent = None, action = problem.actions(problem.initial), path_cost = 0)])  # FIFO queue

    while frontier:
        # and len(frontier) < 50
        #print(f'{frontier} \n')
        node = frontier.popleft()
        if problem.goal == node.state:
            return node
        frontier.extend(node.expand(problem))
        try:
            frontier.remove(node.parent)
        except ValueError:
            continue
        
    return None

In [6]:
#This will not run in a reasonable amount of time.
#keyboard interrupt when tired of waiting.

#Specific instance of maze problem given in the jpg file:

walls = [(9,6),(9,7),(9,8),(9,9),(9,10),(10,10),(10,11),(10,12),(10,13),(9,14),(12,10),(12,11),(13,11),(13,10)]
mazeproblem = Maze( 23  , 15 , (10,8) , (9,11))
mazeproblem.walls = walls

breadth_first_tree_search(mazeproblem)

KeyboardInterrupt: 

Analysis of Breadth First Search Algorithm:

The Breadth First Search Algorithm is extremely inefficient because it revisits the same node again and again. It could not return a solution in a reasonable amount of time. The branching factor of each node is 3 (if the node.state is not adjacent to a wall). By inspection, the solution node has depth 14, so we can expand up to 3^14 nodes to reach the solution (which cannot be done in a reasonable amount of time). This algorithm has exponential time and space complexity.

We can resolve this issue by keeping track of which states (positions in the maze) that the agent has already visited, and do not expand any nodes corresponding to those states. Delete these nodes from the frontier deque immediately. This algorithm is called Breadth First Graph Search, and we implement it below:



In [185]:
def breadth_first_graph_search(problem):


    frontier = deque([Node(problem.initial, parent = None, action = problem.actions(problem.initial), path_cost = 0)])  # FIFO queue
    explored = set()
    
    while frontier:
        # and len(frontier) < 50
        #print(f'{frontier} \n')
        node = frontier.popleft()
        explored.add(node.state)
        
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal == child.state:
                    return child
                frontier.append(child)
    
    #if we reach here, it means that we have visited all reachable states, and none of them was the goal state
    return 'Solution not found'

The next function returns a list of moves to be taken and the path cost of the moves, when given a maze problem. The moves are found using the Breadth First Graph Search algorithm. Our instance of the maze will be called 'mazeproblem', with class 'Maze' and superclass 'Problem'.

In [186]:
def bfgsSolution(problem):
    solution = breadth_first_graph_search(problem)
    if solution == 'Solution not found':
        return solution
    output = []
    current = solution
    while current != problem.initial and current != None:
        output.append(current.action)
        current = current.parent
    output.remove(output[-1])
    output.reverse()
    output.append(solution.path_cost)
    return output


In [187]:
#We remind ourselves again what the problem is:

walls = [(9,6),(9,7),(9,8),(9,9),(9,10),(10,10),(10,11),(10,12),(10,13),(9,14),(12,10),(12,11),(13,11),(13,10)]
mazeproblem = Maze( 23  , 15 , (10,8) , (9,11))
mazeproblem.walls = walls

By inspection, the shortest path has length=12 and path cost=39. The path is hugging the bottom side of the wall and going around it.

In [188]:
bfgsSolution(mazeproblem)

['DOWN',
 'DOWN',
 'DOWN',
 'LEFT',
 'LEFT',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'RIGHT',
 39]

This produces the same path as the one we found by visual inspection. So the algorithm is working correctly. Since this algorithm has cycle check, it does not visit the same state twice.
Analysis:

$\textbf{Worst case for Time Complexity:}$ The algorithm visits every square on the maze map before arriving at the goal. This happens when the initial and goal states are on opposite corners of the map, and the walls are arranged in a way that forces the agent to take a snake like path.

Conclusion: The worst case time-complexity is linear in number of squares that can be visited.

$\textbf{Worst case for Space Complexity:}$ The main memory usage are the deque data structure used to store the frontier, and the set data structure used to store visited nodes. The size of set will never exceed the number of visitable squares in the maze. It can reach the maximum size if all squares are visited before reaching goal state, so the worst case space complexity is at least linear in number of squares that can be visited.

The deque data structure keeps track of the frontier, and the frontier is always a subset of the set of all squares in the maze, so deque cannot have a higher space complexity than linear.

Conclusion: The worst case space-complexity is linear in number of visitable squares.

$\textbf{Completeness:}$ Since the algorithm allows the agent to visit every single reachable state, the algorithm is guaranteed to find a solution when there is one. It will also correctly report failure when no solution exists.

Conclusion: Algorithm is complete.

$\textbf{Cost optimality:}$ From the path costs given (10 for horizontal movement, 1 for vertical movement) it is clear that we want to minimize horizontal movement. The path cost for the solution given is 39 with only 3 horizontal moves, the smallest possible. However, we cannot guarantee cost optimality for all instances of the Maze problem (possibly with different wall configurations, initial and goal states) since we did not account for costs when expanding nodes.

Conclusion: Algorithm is cost optimal for this instance of maze. Cost optimality not guaranteed for all instances.

Next, we examine the Depth First Search with Cycle-Check algorithm, otherwise known as Depth First Graph Search in the notes. The code is given below:

In [189]:
def depth_first_graph_search(problem):

    frontier = [Node(problem.initial, parent = None, action = problem.actions(problem.initial), path_cost = 0)]  # Stack

    explored = set()
    while frontier:
        node = frontier.pop()
        explored.add(node.state)
        
        if problem.goal == node.state:
            return node
        
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
        
        
    return 'Solution not found'


def dfgsSolution(problem):
    solution = depth_first_graph_search(problem)
    if solution == 'Solution not found':
        return solution
    output = []
    current = solution
    while current != problem.initial and current != None:
        output.append(current.action)
        current = current.parent
    output.remove(output[-1])
    output.reverse()
    output.append(solution.path_cost)
    return output

In [190]:
dfgsSolution(mazeproblem)

['UP',
 'RIGHT',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'LEFT',
 'LEFT',
 'LEFT',
 'LEFT',
 'LEFT',
 'LEFT',
 'LEFT',
 'LEFT',
 'LEFT',
 'LEFT',
 'LEFT',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'RIGHT',
 'RIGHT',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'RIGHT',
 'RIGHT',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'DOWN',
 'RIGHT',
 'RIGHT',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'RIGHT',
 'RIGHT',
 'DOWN',
 'DOWN',
 'RIGHT',
 273]

This produces a path of with cost = 461, so it is clearly not the optimal path. The path is going right all the way to the right-boundary,, move up 5 squares, move left till we hit the wall, move up 3 squares to the top-boundary, move left to the left-boundary, move down 4 squares, then move right to goal.

While this algorithm also has cycle check, it is still not optimal.
Analysis:

$\textbf{Worst case for Time Complexity:}$ The algorithm visits every square on the maze map before arriving at the goal. This happens when the initial and goal states are on opposite corners of the map, and the walls are arranged in a way that forces the agent to take a snake like path.

Conclusion: The worst case time-complexity is linear in number of squares that can be visited.

$\textbf{Worst case for Space Complexity:}$ The main memory usage are the list data structure used to store the frontier, and the set data structure used to store visited nodes. The size of set will never exceed the number of squares in the maze. It can reach the maximum size if all squares are visited before reaching goal state, so the worst case space complexity is at least linear in number of squares that can be visited.

The list data structure keeps track of the frontier, and the frontier is always a subset of the set of all squares in the maze, so list cannot have a higher space complexity than linear.

Conclusion: The worst case space-complexity is linear in number of squares that can be visited.

$\textbf{Completeness:}$ Since the algorithm allows the agent to visit every single reachable state, the algorithm is guaranteed to find a solution when there is one. It will also correctly report failure when no solution exists.

Conclusion: Algorithm is complete.

$\textbf{Cost optimality:}$ Algorithm is clearly not cost optimal. We have already found a path with cost = 39 using Breadth First Graph Search. So our new path here with cost = 461 is clearly not cost optimal.

Conclusion: Algorithm is not cost optimal.

$\textbf{A* Search and Greedy Best-first Search}$:
We now examine the Greedy Best-first Search and A* Search algorithms with the 2 heuristics we have defined above.

The first heuristic $h1$ is based on L1 Norm, or Taxicab distance from state to goal. Given two points (a,b) and (x,y), the taxicab distance between (a,b) and (x,y) is the sum of the absolute value of the difference of each coordinate. i.e. $$L1Distance((a,b) , (x,y)) = |a-x| + |b-y|$$.


The second heuristic $h2$ is based on L-Infinity Norm, from state to goal. Given two points (a,b) and (x,y), the L-Infinity Norm distance between (a,b) and (x,y) is the maximum of the absolute value of the difference of each coordinate. i.e. $$LInfDistance((a,b) , (x,y)) = max(|a-x| , |b-y|)$$.

We first define the data structures and helper functions needed:


In [191]:
import heapq

In [192]:
class PriorityQueue:
    """A Queue in which the minimum (or maximum) element (as determined by f and
    order) is returned first.
    If order is 'min', the item with minimum f(x) is
    returned first; if order is 'max', then it is the item with maximum f(x).
    Also supports dict-like lookup."""

    def __init__(self, order='min', f=lambda x: x):
        self.heap = []
        if order == 'min':
            self.f = f
        elif order == 'max':  # now item with max f(x)
            self.f = lambda x: -f(x)  # will be popped first
        else:
            raise ValueError("Order must be either 'min' or 'max'.")

    def append(self, item):
        """Insert item at its correct position."""
        heapq.heappush(self.heap, (self.f(item), item))

    def extend(self, items):
        """Insert each item in items at its correct position."""
        for item in items:
            self.append(item)

    def pop(self):
        """Pop and return the item (with min or max f(x) value)
        depending on the order."""
        if self.heap:
            return heapq.heappop(self.heap)[1]
        else:
            raise Exception('Trying to pop from empty PriorityQueue.')

    def __len__(self):
        """Return current capacity of PriorityQueue."""
        return len(self.heap)

    def __contains__(self, key):
        """Return True if the key is in PriorityQueue."""
        return any([item == key for _, item in self.heap])

    def __getitem__(self, key):
        """Returns the first value associated with key in PriorityQueue.
        Raises KeyError if key is not present."""
        for value, item in self.heap:
            if item == key:
                return value
        raise KeyError(str(key) + " is not in the priority queue")

    def __delitem__(self, key):
        """Delete the first occurrence of key."""
        try:
            del self.heap[[item == key for _, item in self.heap].index(True)]
        except ValueError:
            raise KeyError(str(key) + " is not in the priority queue")
        heapq.heapify(self.heap)


In [193]:
def memoize(fn, slot=None, maxsize=32):
    """Memoize fn: make it remember the computed value for any argument list.
    If slot is specified, store result in that slot of first argument.
    If slot is false, use lru_cache for caching the values."""
    if slot:
        def memoized_fn(obj, *args):
            if hasattr(obj, slot):
                return getattr(obj, slot)
            else:
                val = fn(obj, *args)
                setattr(obj, slot, val)
                return val
    else:
        @functools.lru_cache(maxsize=maxsize)
        def memoized_fn(*args):
            return fn(*args)

    return memoized_fn

We then define and run the algorithms:

In [194]:
def best_first_graph_search(problem, f, display=False):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            if display:
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return 'Solution not found'

In [195]:
def gbfsSolution(problem, f):
    solution = best_first_graph_search(problem, f)
    if solution == 'Solution not found':
        return solution
    output = []
    current = solution
    while current != problem.initial and current != None:
        output.append(current.action)
        current = current.parent
    output.remove(output[-1])
    output.reverse()
    output.append(solution.path_cost)
    return output

In [196]:
gbfsSolution(mazeproblem, mazeproblem.h1)

['DOWN',
 'DOWN',
 'DOWN',
 'LEFT',
 'LEFT',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'RIGHT',
 39]

In [197]:
gbfsSolution(mazeproblem, mazeproblem.h2)

['DOWN',
 'DOWN',
 'DOWN',
 'LEFT',
 'LEFT',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'RIGHT',
 39]

In [198]:
def astar_search(problem, h, display=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n), display)

In [199]:
def astar_search_Solution(problem, h):
    solution = astar_search(problem, h, display=False)
    if solution == 'Solution not found':
        return solution
    output = []
    current = solution
    while current != problem.initial and current != None:
        output.append(current.action)
        current = current.parent
    output.remove(output[-1])
    output.reverse()
    output.append(solution.path_cost)
    return output

In [200]:
astar_search_Solution(mazeproblem, mazeproblem.h1)

['DOWN',
 'DOWN',
 'DOWN',
 'LEFT',
 'LEFT',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'RIGHT',
 39]

$\textbf{Worst case for Time Complexity:}$ Both algorithsm can visit every square on the maze map before arriving at the goal. This happens when the initial and goal states are on opposite corners of the map, and the walls are arranged in a way that forces the agent to take a snake like path.

Conclusion: For both algorithms, the worst case time-complexity is linear in number of squares that can be visited.

$\textbf{Worst case for Space Complexity:}$ The main memory usage are the priority queue data structure used to store the frontier, and the set data structure used to store visited nodes. The size of set will never exceed the number of squares in the maze. It can reach the maximum size if all squares are visited before reaching goal state, so the worst case space complexity is at least linear in number of squares that can be visited.

The priority queue data structure keeps track of the frontier, and the frontier is always a subset of the set of all squares in the maze, so priority queue cannot have a higher space complexity than linear.

Conclusion: For both algorithms, the worst case space-complexity is linear in number of squares that can be visited.

$\textbf{Completeness:}$ Both algorithms can visit all possible squares, they only prioritize certain squares according to their defined behaviour and the heuristics h1 and h2. Hence, both algorithms are complete. They will also report failure to find solution if that is the case.

Conclusion: Both algorithms are complete.

$\textbf{Cost optimality:}$ Both algorithms returned the optimal path in this instance.

Conclusion: Both algorithm are cost optimal.

$\textbf{Uniform Cost Search and Iterative Deepening Search with Cycle Check}$

We now examine these two algorithms.

In [201]:
#Uniform Cost Search

def uniform_cost_search(problem):
    
    return best_first_graph_search(problem, lambda n: n.path_cost)

In [202]:
uniform_cost_search(mazeproblem)

<Node (9, 11)>

In [203]:
def uniform_cost_search_solution(problem):
    solution = uniform_cost_search(problem)
    if solution == 'Solution not found':
        return solution
    output = []
    current = solution
    while current != problem.initial and current != None:
        output.append(current.action)
        current = current.parent
    output.remove(output[-1])
    output.reverse()
    output.append(solution.path_cost)
    return output

In [204]:
uniform_cost_search_solution(mazeproblem)

['DOWN',
 'DOWN',
 'DOWN',
 'LEFT',
 'LEFT',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'RIGHT',
 39]

In [205]:
def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)


In [206]:
def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    '''
    LIFOQueue = list
    '''
    frontier = [Node(problem.initial)]
    
    result = 'failure'
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif node.depth >= limit:
            result = 'Solution not found'
        elif not is_cycle(node):
            for child in node.expand(problem):
                frontier.append(child)
    return result

In [207]:
def iterative_deepening_search(problem, maxdepth = 50):
    "Do depth-limited search with increasing depth limits."
    for i in range(1, maxdepth + 1):
        result = depth_limited_search(problem, limit = i)
        if result != 'Solution not found' and result != 'failure':
            return result
    return 'Solution not found within given maximum depth'

In [208]:
depth_limited_search(mazeproblem, limit = 12)

<Node (9, 11)>

In [209]:
def iterative_deepening_search_solution(problem, maxlimit = 10):
    solution = iterative_deepening_search(problem , maxdepth = maxlimit)
    if solution == 'Solution not found':
        return solution
    output = []
    current = solution
    while current != problem.initial and current != None:
        output.append(current.action)
        current = current.parent
    output.remove(output[-1])
    output.reverse()
    output.append(solution.path_cost)
    return output

In [210]:
iterative_deepening_search(mazeproblem, maxdepth = 12)

<Node (9, 11)>

In [211]:
iterative_deepening_search_solution(mazeproblem, maxlimit = 12)

['DOWN',
 'DOWN',
 'DOWN',
 'LEFT',
 'LEFT',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'UP',
 'RIGHT',
 39]

$\textbf{Worst case for Time Complexity:}$ Both algorithms expand nodes in an order that makes the agent visit every square on the maze map before arriving at the goal. This worst case can be achieved: by setting the initial and goal states on opposite corners of the map, and arrange the walls in a way that forces the agent to take a snake like path through all the squares.

Moreover, iterative deepening search can expand the top level nodes repeatedly (via depth limited search) for each iteration. The worst case is when the solution is at maximum possible depth, again by having the agent visit all possible squares.

Conclusion: For uniform cost search, the worst case time-complexity is linear in number of squares that can be visited. For iterative deepening search, the worst case time-complexity is quadratic in number of squares that can be visited.

$\textbf{Worst case for Space Complexity:}$ Uniform cost search is effectively best first graph search with a fixed cost function. Hence, it is worst case space complexity is linear in number of squares that can be visited. 

For iterative deepening search, the main memory usage is the list structure keeping track of the frontier. Not every square can be the frontier at a certain depth. Intuitively, the set of squares that can be frontiers at a given depth is the set of squares that have L0-Norm (taxicab metric) distance = depth from the initial starting square. The L0-norm was a heuristic used in the A*-search. Due to properties of the L0-Norm, the maximum number of squares in the frontier scales linearly with the depth. The worst case is achieved when there are no walls and the map is a square. 

Conclusion: The space-complexity for uniform cost search is linear in number of squares that can be visited. The space-complexity for iterative deepening search is linear with respect to the length of the map. (square root of the total number of squares.)

$\textbf{Completeness:}$ Both algorithms can visit all possible squares, hence both algorithms are complete. They will also report failure to find solution if that is the case.

Conclusion: Both algorithms are complete.

$\textbf{Cost optimality:}$ Both algorithms returned the optimal path in this instance. However, only the uniform cost search takes into account the cost. Iterative deepening search can product a non cost-optimal path if that path is at a lower depth than the cost-optimal path. One such possibility is having an L shaped wall with a long vertical component but a short horizontal one. Set the starting point at the corner of the L shaped wall, and the goal at the corner on the other side. Moving vertically around the top of the L shaped wall produces a longer path but smaller cost as compared to going horizontally to the side. This is because horizontal movements incur 10-times the cost compared to vertical movements.

Uniform cost search is always cost optimal because it always makes moves that incurs a lower total cost first before making higher cost moves.

Conclusion: Both algorithm are cost optimal in this instance, but there are instances where iterative deepening search is not. Uniform cost search is always cost optimal.